In [3]:
# Select Top 400 images from coco validation set by ranking the top-3 Clip scopre with the captions.

In [4]:
%load_ext autoreload
%autoreload 2

import os
from PIL import Image, ImageOps
import requests
import torch
import matplotlib.pyplot as plt
import numpy as np
import json

path_to_coco = "./coco"
# coco captions
with open(path_to_coco + "/annotations/captions_val2017.json", "r") as f:
    caption = json.load(f)
    print(caption.keys())

print(len(caption['images']))

from torchmetrics.multimodal import CLIPScore
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.PILToTensor()
])

metric = CLIPScore(model_name_or_path=path_co_clip)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
dict_keys(['info', 'licenses', 'images', 'annotations'])
dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])
5000


Using TensorFlow backend.


In [5]:
image_paths = []
image_captions = []
image_caption_scores = []

for i in range(len(caption['images'])):
    if i % 50 == 0:
        print(i)
    select_image = i
    captions = []
    scores = []
    image_path = path_to_coco + "/images/val2017/" + caption['images'][select_image]['file_name']
    init_image = Image.open(image_path).convert('RGB')
    # map image with its caption 
    for j in range(len(caption['annotations'])):
        if caption['annotations'][j]['image_id'] == caption['images'][select_image]['id']:
            captions.append(caption['annotations'][j]['caption'])
            # compute clip score between the original image with its caption
            score_clip = metric(transform(init_image), caption['annotations'][j]['caption'])
            scores.append(score_clip.detach().numpy())
    image_paths.append(image_path)
    scores = np.array(scores)
    scores_idx = np.argsort(scores)
    scores_select = []
    captions_select = []
    # only consider the top-3 captions
    for idx in scores_idx[-3:]:
        scores_select.append(scores[idx])
        captions_select.append(captions[idx])
    image_caption_scores.append(np.mean(np.array(scores_select)))
    image_captions.append(captions_select)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950


In [6]:
# select top 400 images
image_paths_final = []
image_captions_final = []

score_rank = np.argsort(image_caption_scores)
for idx in score_rank[-400:]:
    image_paths_final.append(image_paths[idx])
    image_captions_final.append(image_captions[idx])

In [7]:
#store selected images and captions
import json
caption_dict = {}
image_folder = "images_all/"
for i in range(len(image_paths_final)):
    init_image = Image.open(image_paths_final[i]).convert('RGB')
    image_name = image_paths_final[i].split('/')[-1]
    save_path = image_folder + image_name
    caption_dict[image_name] = image_captions_final[i]
    init_image.save(save_path)
with open("captions.json", "w") as f:
    json.dump(caption_dict, f)
